In [1]:
from sklearn.datasets import fetch_olivetti_faces
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np

## Olivetti Faces
Nosso dataset é composto de 10 fotos de rostos de 40 diferentes pessoas, totalizando 400 imagens de 64x64 píxeis cada.

### Topologia da Rede Neural
A camada de input é composta por 4096 nodos, onde são injetados os píxeis da imagem de entrada.
A camada de output é composta por 40 nodos, indicando a probabilidade de combinação com o rosto de uma determinada pessoa dentre as 40.
Durante nossos testes percebemos que o ganho 

In [ ]:
faces = fetch_olivetti_faces()

In [3]:
data = faces.data
target = faces.target

Para o dataset escolhido não há necessidade de pré-processamento, as imagens já são recebidas do mesmo tamanho, no mesmo formato, e em escala de cinza.

### Validação Cruzada
Para que seja feita validação cruzada, precisamos seccionar nosso dataset em dois pedaços: um para treinamento da rede e outro para avaliação do treinamento. Utilizando um método do SciKit chamado `train_test_split` conseguimos gerar essas duas partições automaticamente, o mesmo tempo que os dados originais são misturados para garantir o treinamento de todos as possíveis saídas.

Após tentativas com diferentes tamanhos de secções do dataset para treino da rede, chegamos ao número 80% como o limite: utilizando mais do que essa quantidade não percebemos melhoria na avaliação da rede durante a fase de testes.

In [32]:
data_train, data_test, target_train, target_test = train_test_split(data, target, test_size=0.2, random_state=0)

print("Treinamento: \t" + str(len(target_train)))
print("Teste: \t\t" + str(len(target_test)))
print("Total: \t\t" + str(len(faces.target)))

Treinamento: 	320
Teste: 		80
Total: 		400


### Escolha da Rede

Por se tratar de um problema de classificação de uma dada imagem em uma das 40 possíveis classes de `target`,  instanciamos uma Rede Neural classificadora **Perceptron Multi-Camadas**, composta por uma camada de entrada de 4096 nodos, uma camada intermediária (hidden) de 2048 nodos, e uma camada de saída de 40 nodos.

Optamos por utilizar a função de ativação de retificação linear (`relu`, ou Hard Max) por ter demonstrado melhores resultados durante nossos testes, e por ter sido reconhecida como a mais rápida e que obtém melhores resultados em treinamento não guiado de redes neurais aplicadas a imagens.

In [23]:
clf = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(2048), activation='relu')
clf.fit(data_train, target_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=2048, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

Após ter sido executado o treinamento da rede, utilizamos o pedaço do dataset selecionado para testes.

In [24]:
target_result = clf.predict(data_test)

### Avaliação dos Resultados
Para quantificarmos os acertos da rede, utilizamos o seguinte trecho recursivo de código que verifica quantos valores iguais existem nos mesmos índices de dois dados arrays.

In [33]:
def matches(arr1, arr2):
    try:
        return (1 if arr1[0]==arr2[0] else 0) + matches(arr1[1:], arr2[1:])
    except IndexError:
        return 0

Utilizamos, então, a nossa função `match` sobre os resultados recebidos da rede para nossa porção de testes do dataset.

In [34]:
matchCount = matches(target_result, target_test)
print(float(matchCount)/float(len(target_result)))

0.9125


A rede obteve pouco mais de 91% de acertos.